In [2]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

### Step 1: Create a soup object from the home page

In [3]:
url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
# Previewing what 'soup' looks like
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Nutrition Information</title>
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>
</head>
<body>
<header>
<section class="container">
<nav class="navbar navbar-expand-lg navbar-light bg-light" role="navigation">
<a class="navbar-brand" href="/">Nutrition Information</a> </nav>
</section>
</header>
<main class="container" role="main">
<br/>
<div class="alert alert-danger">
        NOTE: This data is super old and rife with errors. It's meant for scraping practice only.
      </div>
<table class="table" id="restaurants">
<thead>
<tr>
<th>Name</th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<td>
<a href="restaurants/1.html">A&amp;W Restauran

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [8]:
# Isolate the table that has all the restaurant links
restaurants_table = soup.find('table', {'id': 'restaurants'})

# Create an empty list
restaurants = []

# Loop through each link in the restaurants table
for restaurant_link in restaurants_table.find_all('a'):
    # Start with an empty dictionary
    restaurant = {}
    
    # Add name
    restaurant['name'] = restaurant_link.text
    
    # Add href
    restaurant['href'] = restaurant_link['href']
    
    # Add restaurant to our list of restaurants
    restaurants.append(restaurant)

restaurants[:5]

[{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'},
 {'name': "Applebee's", 'href': 'restaurants/2.html'},
 {'name': "Arby's", 'href': 'restaurants/3.html'},
 {'name': 'Atlanta Bread Company', 'href': 'restaurants/4.html'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'href': 'restaurants/5.html'}]

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [9]:
test_url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/1.html'
test_res = requests.get(test_url)
test_soup = BeautifulSoup(test_res.content, 'lxml')

print(test_soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Nutrition Information</title>
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>
</head>
<body>
<header>
<section class="container">
<nav class="navbar navbar-expand-lg navbar-light bg-light" role="navigation">
<a class="navbar-brand" href="/">Nutrition Information</a> </nav>
</section>
</header>
<main class="container" role="main">
<br/>
<div class="alert alert-danger">
        NOTE: This data is super old and rife with errors. It's meant for scraping practice only.
      </div>
<h3 class="display-3">
A&amp;W Restaurants</h3>
<p class="lead">Data Source: http://www.awrestaurants.com/</p>
<table class="table">
<thead>
<tr>
<th>Name<

In [10]:
# Start with an empty list
foods = []

# Loop through each restaurant in the previous step
for restaurant in restaurants:
    print('Scraping {}'.format(restaurant['name']))
    
    href = restaurant['href']
    restaurant_url = f'https://pages.git.generalassemb.ly/rldaggie/for-scraping/{href}'
    
    # Use requests library to get the content from each restaurant page
    restaurant_res = requests.get(restaurant_url)
    
    # Create soup object from restauarant html
    restaurant_soup = BeautifulSoup(restaurant_res.content, 'lxml')
    
    # Isolate the foods table from restaurant page
    table = restaurant_soup.find('table')
    
    # Loop through each row in the tbody of the restaurants table
    for row in table.find('tbody').find_all('tr'):
        # We'll use almost all the <td /> tags for each row, might as well create a variable
        cells = row.find_all('td')
        
        # Start with an empty food dictionary
        food = {}
        
        # Add the restaurant's name (No need for the slug, that was just for scraping purposes)
        food['restaurant'] = restaurant['name']
        
        # Add food name from firs cell
        food['name'] = cells[0].text
        
        # Add category, note the .strip() for removing white space
        food['category'] = cells[1].text.strip()
        
        # Add calories 
        food['calories'] = cells[2].text
        
        # Add fat
        food['fat'] = cells[3].text
        
        # Add carbs
        food['carbs'] = cells[4].text
        
        # Add the food dictionary to our list of foods
        foods.append(food)
        
    # Be courteous and throttle your scrapes!
    time.sleep(3)

Scraping A&W Restaurants
Scraping Applebee's
Scraping Arby's
Scraping Atlanta Bread Company
Scraping Bojangle's Famous Chicken 'n Biscuits
Scraping Buffalo Wild Wings
Scraping Burger King
Scraping Captain D's
Scraping Carl's Jr.
Scraping Charley's Grilled Subs
Scraping Chick-fil-A
Scraping Chili's
Scraping Chipotle Mexican Grill
Scraping Church's
Scraping Corner Bakery Cafe
Scraping Dairy Queen
Scraping Denny's
Scraping El Pollo Loco
Scraping FATZ
Scraping Fazoli's
Scraping Five Guys Burgers and Fries
Scraping Golden Chick
Scraping Hardee's
Scraping IHOP
Scraping In-N-Out Burger
Scraping Jack in the Box
Scraping Jimmy Johns
Scraping Joe's Crab Shack
Scraping KFC
Scraping McDonald's
Scraping O'Charley's
Scraping Olive Garden
Scraping Outback Steakhouse
Scraping Panda Express
Scraping Panera Bread
Scraping Popeye's
Scraping Quiznos
Scraping Red Robin Gourmet Burgers
Scraping Romano's Macaroni Grill
Scraping Ruby Tuesday
Scraping Subway
Scraping Taco Bell
Scraping Taco Bueno
Scraping Wend

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [11]:
df = pd.DataFrame(foods)
df.shape

(5131, 6)

In [12]:
df.head()

,restaurant,name,category,calories,fat,carbs
0,A&W Restaurants,Original Bacon Double Cheeseburger,Burgers,760,45,45
1,A&W Restaurants,Coney (Chili) Dog,Entrees,340,20,26
2,A&W Restaurants,Chili Fries,French Fries,370,15,49
3,A&W Restaurants,Strawberry Milkshake (small),Shakes,670,29,90
4,A&W Restaurants,A&W® Root Beer Freeze (large),Shakes,820,18,150


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [13]:
df.to_csv('foods.csv', index=False)